In [3]:
###

In [4]:
### Import Libaries 

In [5]:
import requests
import pandas as pd
import json
import numpy as np

In [6]:
def book_odds(date):
    
    '''
    date => eg;20221018
    '''
    
    url = 'https://api.actionnetwork.com/web/v1/scoreboard/nba?bookIds=15,30,76,75,123,69,68,972,71,247,79&date=' + date
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:101.0) Gecko/20100101 Firefox/101.0'}
    
    r = requests.get(url, headers=headers)

    tb = json.loads(r.content.decode('utf-8'))
    
    games = tb['games']
    
    game_odds = []
    
    for i in games:
        odds = i['odds']
        
        dfs = []
        
        for o in odds:
            dfs.append(pd.DataFrame([o]))
        df = pd.concat(dfs)    
        
        t1 = pd.DataFrame([i['teams'][0]])
        t2 = pd.DataFrame([i['teams'][1]])
    
        t1 = t1[['id','full_name','display_name','short_name','location','abbr', 'standings']].add_suffix('_home')
        t2 = t2[['id','full_name','display_name','short_name','location','abbr', 'standings']].add_suffix('_away')
        
        t = pd.concat([t1,t2], axis = 1)
        dt =pd.concat([t,df], axis = 1)
        
        game_odds.append(dt)    
        
    df = pd.concat(game_odds)      
    
    book_ids = {15:'open', 30: 'best odds', 76:'Pointsbet',75:'betmgm',  123:'caesars', 69:'fanduel', 68:'draftking', 972:'betrivers', 71: 'playsugarhouse', 274:'unibet', 79: 'bet365' }
    
    df['bookie'] = df['book_id'].map(book_ids)
    
    df['bookie_date'] = date
    
    df['bet_date_id'] = df['abbr_home'] +  df['bookie_date']

    return df  

In [7]:
bt = book_odds('20221018')

## **Filter by open line and best odd bookie**

In [8]:
btf = bt[bt['type'] == 'game']
btf = btf[btf['book_id'] == 30]

In [9]:
btf.head()

,id_home,full_name_home,display_name_home,short_name_home,location_home,abbr_home,standings_home,id_away,full_name_away,display_name_away,short_name_away,location_away,abbr_away,standings_away,ml_away,ml_home,spread_away,spread_home,spread_away_line,spread_home_line,over,under,draw,total,away_total,away_over,away_under,home_total,home_over,home_under,ml_home_public,ml_away_public,spread_home_public,spread_away_public,total_under_public,total_over_public,ml_home_money,ml_away_money,spread_home_money,spread_away_money,total_over_money,total_under_money,meta,num_bets,book_id,type,inserted,line_status,bookie,bookie_date,bet_date_id
0,164,Boston Celtics,Celtics,Celtics,Boston,BOS,"{'win': 11, 'loss': 9, 'ties': None, 'overtime...",163,Philadelphia 76ers,76ers,76ers,Philadelphia,PHI,"{'win': 7, 'loss': 4, 'ties': None, 'overtime_...",150,-186,4,-4,-110,-110,-108,-108,None,213.5,104.5,-130,-100,108.5,-115,-115,34,66,63,37,59,41,24,76,2,98,97,3,None,0,30,game,2022-10-15T20:17:00.845694+00:00,"{'over': 0, 'under': 0, 'ml_away': 0, 'ml_draw...",best odds,20221018,BOS20221018
0,186,Los Angeles Lakers,Lakers,Lakers,Los Angeles,LAL,"{'win': 8, 'loss': 8, 'ties': None, 'overtime_...",182,Golden State Warriors,Warriors,Warriors,Golden State,GSW,"{'win': 6, 'loss': 7, 'ties': None, 'overtime_...",200,-250,6,-6,-111,-109,-108,-111,None,227.5,110.5,-130,-100,116.5,-115,-115,69,31,78,22,50,50,50,50,46,54,3,97,None,0,30,game,2022-10-17T13:51:35.277514+00:00,"{'over': 0, 'under': 0, 'ml_away': 0, 'ml_draw...",best odds,20221018,LAL20221018


## **Get NBA Game Logs by Season**

In [30]:

def game_logs(season):
    # season: '2022-23'
    
    url = 'https://stats.nba.com/stats/leaguegamelog'
    headers= {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36', 'Referer': 'https://www.nba.com/'}
    payload = {
        'Counter': '1000',
        'DateFrom': '',
        'DateTo': '',
        'Direction': 'DESC',
        'LeagueID': '00',
        'PlayerOrTeam': 'T',
        'Season': season,
        'SeasonType': 'Regular Season',
        'Sorter': 'DATE'}
    
    jsonData = requests.get(url, headers=headers, params=payload).json()
    
    rows = jsonData['resultSets'][0]['rowSet']
    columns = jsonData['resultSets'][0]['headers']
    
    df = pd.DataFrame(rows, columns=columns)
    
    df['DATE'] = pd.to_datetime(df['GAME_DATE'])
    df['bookie_date'] = df['DATE'].dt.strftime('%Y%m%d')
    df['home'] = np.where(df['MATCHUP'].str.contains('@'), 0, 1)
    del df['SEASON_ID']
    
    df['bet_date_id'] = df['TEAM_ABBREVIATION'] + df['bookie_date']
    
    return df 

In [31]:
tm = game_logs('2022-23')    

In [32]:
# Group the DataFrame based on 'group_column'
grouped = tm.groupby('GAME_ID')

# Create a list of DataFrames, one for each group
grouped_dataframes = [group for _, group in grouped]

tm1 = grouped_dataframes[0]

In [33]:
len(list(tm1))

32

In [23]:
tm1 = tm1.sort_values(by='home', ascending=False)
tm1 = tm1.assign(**{f'away_{column}': value for column, value in tm1.iloc[1].items()})
tm1 = tm1.head(1).reset_index(drop= True)

In [24]:
## Drop duplicate columns 

In [25]:
# Step 1: transpose the DataFrame to compare columns instead of rows
tm1_transpose = tm1.T

# Step 2: Use the 'duplicated()' function to find duplicate columns (similar values)
duplicate_columns = tm1_transpose[tm1_transpose.duplicated()]

# Step 3: Drop the duplicate columns from the original DataFrame
tm1 = tm1.drop(columns=duplicate_columns.index)

In [26]:
tm1.head()

,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,PLUS_MINUS,VIDEO_AVAILABLE,DATE,bookie_date,bet_date_id,away_TEAM_ID,away_TEAM_ABBREVIATION,away_TEAM_NAME,away_MATCHUP,away_WL,away_FGM,away_FGA,away_FG_PCT,away_FG3M,away_FG3A,away_FG3_PCT,away_FT_PCT,away_OREB,away_DREB,away_REB,away_AST,away_TOV,away_PF,away_PTS,away_PLUS_MINUS,away_home,away_bet_date_id
0,1610612738,BOS,Boston Celtics,0022200001,2022-10-18,BOS vs. PHI,W,240,46,82,0.561,12,35,0.343,22,28,0.786,6,30,36,24,8,3,11,126,9,1,2022-10-18,20221018,BOS20221018,1610612755,PHI,Philadelphia 76ers,PHI @ BOS,L,40,80,0.5,13,34,0.382,0.857,4,27,31,16,14,25,117,-9,0,PHI20221018


In [27]:
btf = btf[btf['bet_date_id'] == tm1['bet_date_id'][0]]
btf.head()

,id_home,full_name_home,display_name_home,short_name_home,location_home,abbr_home,standings_home,id_away,full_name_away,display_name_away,short_name_away,location_away,abbr_away,standings_away,ml_away,ml_home,spread_away,spread_home,spread_away_line,spread_home_line,over,under,draw,total,away_total,away_over,away_under,home_total,home_over,home_under,ml_home_public,ml_away_public,spread_home_public,spread_away_public,total_under_public,total_over_public,ml_home_money,ml_away_money,spread_home_money,spread_away_money,total_over_money,total_under_money,meta,num_bets,book_id,type,inserted,line_status,bookie,bookie_date,bet_date_id
0,164,Boston Celtics,Celtics,Celtics,Boston,BOS,"{'win': 11, 'loss': 9, 'ties': None, 'overtime...",163,Philadelphia 76ers,76ers,76ers,Philadelphia,PHI,"{'win': 7, 'loss': 4, 'ties': None, 'overtime_...",150,-186,4,-4,-110,-110,-108,-108,None,213.5,104.5,-130,-100,108.5,-115,-115,34,66,63,37,59,41,24,76,2,98,97,3,None,0,30,game,2022-10-15T20:17:00.845694+00:00,"{'over': 0, 'under': 0, 'ml_away': 0, 'ml_draw...",best odds,20221018,BOS20221018


In [28]:
tm1.head()

,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PTS,PLUS_MINUS,VIDEO_AVAILABLE,DATE,bookie_date,bet_date_id,away_TEAM_ID,away_TEAM_ABBREVIATION,away_TEAM_NAME,away_MATCHUP,away_WL,away_FGM,away_FGA,away_FG_PCT,away_FG3M,away_FG3A,away_FG3_PCT,away_FT_PCT,away_OREB,away_DREB,away_REB,away_AST,away_TOV,away_PF,away_PTS,away_PLUS_MINUS,away_home,away_bet_date_id
0,1610612738,BOS,Boston Celtics,0022200001,2022-10-18,BOS vs. PHI,W,240,46,82,0.561,12,35,0.343,22,28,0.786,6,30,36,24,8,3,11,126,9,1,2022-10-18,20221018,BOS20221018,1610612755,PHI,Philadelphia 76ers,PHI @ BOS,L,40,80,0.5,13,34,0.382,0.857,4,27,31,16,14,25,117,-9,0,PHI20221018


In [29]:
len(list(tm1))

52